### Exponential Thermal Tensor Network Renormalization Group (XTRG)

Date: <span style = "color: wheat"> 23.07.2025 </span> | Author of the code: <span style = "color: wheat"> Matthias Pawlik, Damiano Aliverti </span>

The Exponential Tensor Renormalization Group (XTRG) is a powerful numerical method for computing the thermal density matrix $\hat{\rho} = e^{− \beta \hat{H}}$ of finite-size quantum systems, where $\beta$ is the inverse temperature and $\hat{H}$ is the many-body Hamiltonian. In this problem, you will implement the XTRG algorithm and use it to compute the partition function of a one-dimensional ($1D$) XY model from high temperatures ($\beta \sim 10^{−6}$) down to low temperatures ($\beta \sim 1$).

<span style = "color: wheat"> [1]  </span> B.-B. Chen, L. Chen, Z. Chen, W. Li, and A. Weichselbaum, Phys. Rev. X 8, 031082 (2018), https://doi.org/10.1103/PhysRevX.8.031082.

<span style = "color: wheat"> (a) </span> Initialize the thermal density matrix as a matrix product operator (MPO) using linear initialization, $\rho(\beta_0) \approx \mathbb{I} − \beta_0 H$, as described in Appendix C.2 of Ref. [1]. Use $\beta_0 = 10^{−6}$ as the initial inverse temperature.

We define the Hamiltonian of the spin-$\frac{1}{2}$ XY-chain of length $\mathcal{L}$ with open boundary conditions (OPC) to $\hat{H} = J \sum_{l = 1}^{\mathcal{L}-1} (\hat{S}^x_l \hat{S}^x_{l+1} + \hat{S}^y_l \hat{S}^y_{l+1}) \equiv J \sum_{l = 1}^{\mathcal{L}-1} (\hat{S}^+_l \hat{S}^-_{l+1} + \hat{S}^-_l \hat{S}^+_{l+1})$.

<span style = "color: SkyBlue"> Solution: </span>

In [1]:
include("source/MPO.jl")
using .MPO

In [2]:
function linear_intialization(L::Int, J::Float64, beta0::Float64=1e-6)

    # Generate the MPO Hamiltonian for the XY-Model
    H_mpo = xychain_mpo(L, J)

    # Construct an MPO representation of the identity operator
    Id_mpo = identity_mpo(L)
    println(size(Id_mpo[end]), size(Id_mpo[2]))

    # Multiply all elements of H with inverse initial temperature
    H_mpo = [-beta0 * W for W in H_mpo]
    println(size(H_mpo[end]), size(H_mpo[2]))
    
    # Add the MPOs and obtain new local tensors with bond dimension D = 4 + 1
    rho0 = add_mpo(H_mpo, Id_mpo)

    println(size(rho0[end]), size(rho0[1]), size(rho0[2]))

    return rho0

end

linear_intialization (generic function with 2 methods)

In [ ]:
# regarding H_mpo = [-beta0 * W for W in H_mpo] from the above cell: i think this means multiplying the Hamiltonian by -beta0^(chain length L) (?). we could do, instead, H_mpo = [c * W for W in H_mpo] with c = log_L(beta0), and incorporate the sign as a parameter of the function add_mpo

<span style = "color: wheat"> (b) </span> Implement the XTRG algorithm following the strategy in Sec. II of Ref.[1]. The key idea is to
iteratively double the inverse temperature, $\rho(2 \beta) = \rho(\beta) \times \rho(\beta)$, by contracting the MPO with itself. After each multiplication, the MPO bond dimension increases and must be truncated. This can be done variationally using DMRG-type sweeping, as detailed in Appendix D of Ref. [1].

<span style = "color: SkyBlue"> Solution: </span>

In [ ]:
function XTRG_step(rho::Vector, beta::Float64)

    beta += beta

    return rho, beta

end

<span style = "color: wheat"> (c) </span> Apply your XTRG implementation to the 1D XY-model of length $L = 10$. Perform $20$ XTRG
steps starting from $\beta_0 = 10^{−6}$, so that the final inverse temperature is $\beta = 2^{20} \beta_0$. At each step, compute the partition function $Z = \text{Tr}(\rho(\beta))$. Compare your numerical results with the analytical solution provided in Appendix F of Ref. [1] over the full temperature range.

<span style = "color: SkyBlue"> Solution: </span>

## Tests

From here on we can write and try out code, to be later wrapped up into functions and "official" cells of this notebook

Here is a possible scheme for implementing `square_mpo`, the function that takes as parameters an `mpo`, an upper bound `Dmax` to the bond dimension, and a number `Nsweeps` of sweeps, and returns an mpo `mpo2` representing the square of the operator represented my `mpo`. See source/square_mpo.jl for better description of what this function `square_mpo` should do. There, the easy case where (bond dimension of `mpo`)^2 <= Dmax is already implemented (no truncation is needed, so easy!). Here let's assume (bond dimension of `mpo`)^2 > Dmax, which requires truncation and actually understanding appendix D of the paper (sigh). This scheme closely follows ideas and in particular the indexing conventions of the lecture function `dmrg_2site`! Having a tab open with the implementation of `dmrg_2site` may help understand the scheme below.

- Get as parameters `mpo` to be squared, `Dmax`, and `Nsweeps`
- Initialize `mpo2` (will be output) as a copy of `mpo` (...any better ideas?) with enlarged bond dimension = `Dmax`. one has to add a bunch of zeros (1)
- by reshaping `mpo2` to an mps (by joining its physical legs), bring it to left-canonical form. Then reshape it back to the mpo form (by disjoining physical legs) (2)
- initialize `Vlr` as `Hlr` in `dmrg_2site`. roughly speaking `Vlr[l + 1]` will contain the environment tensors $V_{L/R}$ (see fig. 17c in the paper) relative to site `l` when the algorithm/sweeping is working at a couple of site that are on the right/left of site `l`.

In [ ]:
Vlr = Array{Any}(undef,1,L+2); # L is the chain length
Vlr[1] = reshape([1],(1,1,1)); # watch out, the dummy site on the left of the first site makes indices shift by one!
Vlr[end] = reshape([1],(1,1,1));

- Fill `Vlr[2], ..., Vlr[L + 1]` with iterative contractions of `mpo`, `mpo` and `mpo2`. For that, we can use a modified version of updateLeft, say `updateLeft_mod` (3), doing what's described in fig. 17c
- Start a for cycle with the index `itS = (1:Nsweeps)`
- RIGHT TO LEFT SWEEP: `for itL = (L:-1:2)`, update (4) as in appendix D `mpo2[itL-1]` and `mpo2[itL]`. This involves `Vlr[itL-1]` as left environment $V_L$ and `Vlr[itL + 2]` as right environment $V_R$. It also involves an SVD (5) to get `mpo2[itL-1]` and `mpo2[itL]` out of the 2-site optimized tensor ($C_i C_{i+1}$ in the paper). In particular, the updated `mpo2[itL]` must be an "MPO-right isometry", so that the updated `mpo2`  will be "MPO-site canonical" with new isometry center at site `itL - 1` (still need to really get the details of all that...). After that, update `Vlr[itL + 1]` using `updateLeft_mod` (and an index permutation since we are "updating leftwards")
- LEFT TO RIGHT SWEEP: similar to above: `for itL = (1:L-1)`, update `mpo2[itL]` and `mpo2[itL+1]` using `Vlr[itL] ` and `Vlr[itL + 3]`. Then update `Vlr[itL + 1]` using `updateLeft_mod`
- end of cycle with index itS
- `mpo2` should now approximately represent the square of the operator represented by `mpo`!! ...for big enough `Dmax` and `Nsweeps`

Some "isolated" functions that we can isolate and test before implementing the whole scheme are the following:

(1) write & test a function to enlarge bond dim. of an MPO by "adding zeros"

(2) understand how to bring an mpo to left-canonical form (what does this really mean? whould we really just transform the mpo into an mps by joining physical legs, getting canonical form of mps, and get back to mpo form?)

(3) write & test `updateLeft_mod` following fig 17c. use/copy `updateLeft`

(4) write & test a function for the two-site optimization (without svd), see appendix D

(5) wirte & test a function to SVD a 2-site, 6-leg MPO tensor. use/copy `svd`, `svdleft` and `svdright` from lecture 